In [ ]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
import json
import requests

# Your service key JSON file relative to this notebook
waste_app_ai_core_key_path = 'waste_app_ai_core_key.json'

# Loads the service key file
with open(waste_app_ai_core_key_path) as ask:
    waste_app_ai_core_key = json.load(ask)

# Get credentials from the service key
uaa_url = waste_app_ai_core_key["url"]
clientid = waste_app_ai_core_key["clientid"]
clientsecret = waste_app_ai_core_key["clientsecret"]
ai_api_url = waste_app_ai_core_key["serviceurls"]["AI_API_URL"]

# Get OAuth token directly
params = {"grant_type": "client_credentials"}
resp = requests.post(f"{uaa_url}/oauth/token",
                    auth=(clientid, clientsecret),
                    params=params)
token = resp.json()["access_token"]
print("✅ Successfully obtained OAuth token")

# Create or verify resource group
resource_group = "default"  # Change this to your preferred resource group name
url = ai_api_url + '/v2/admin/resourceGroups'
payload = json.dumps({
    "resourceGroupId": resource_group
})
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token}'
}

print(f"Attempting to create resource group: {resource_group}")
response = requests.request("POST", url, headers=headers, data=payload)
print(f"Response status: {response.status_code}")
print(f"Response body: {response.text}")

# If successful or if resource group already exists, we can continue with the client
if response.status_code == 201 or response.status_code == 409:  # Created or Already Exists
    print(f"✅ Resource group '{resource_group}' is ready to use")
else:
    print("⚠️ Could not create resource group. Check response for details.")

# Creating an AI API client instance with the verified resource group
aicore_client = AICoreV2Client(
    base_url = ai_api_url + "/v2",  # The present AI API version is 2
    auth_url = uaa_url + "/oauth/token",
    client_id = clientid,
    client_secret = clientsecret
)

print(f"AI Core client initialized. Default resource group: {resource_group}")

In [ ]:
aicore_client.scenario.query('default').count